In [13]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import yaml

In [30]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())


print(launch_command(conf))

export CUDA_VISIBLE_DEVICES=1
python -m vllm.entrypoints.openai.api_server --model hatakeyama-llm-team/Tanuki-8B-Instruct --max-model-len 4000 --port 8000 --gpu-memory-utilization 0.2 
export CUDA_VISIBLE_DEVICES=1
python -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-medium-128k-instruct --max-model-len 4000 --port 8001 --gpu-memory-utilization 0.4 
export CUDA_VISIBLE_DEVICES=0
python -m vllm.entrypoints.openai.api_server --model Rakuten/RakutenAI-7B-chat --max-model-len 4000 --port 8002 --gpu-memory-utilization 0.2 --chat-template templates/RakutenAI-7B-chat.jinja



In [18]:
client_dict=get_client_dict(conf)
client_dict

{'Tanuki-8B-Instruct': {'model': 'hatakeyama-llm-team/Tanuki-8B-Instruct',
  'client': <openai.OpenAI at 0x7ff044236c10>},
 'Phi-3-medium-128k-instruct': {'model': 'microsoft/Phi-3-medium-128k-instruct',
  'client': <openai.OpenAI at 0x7ff0442a3150>},
 'RakutenAI-7B-chat': {'model': 'Rakuten/RakutenAI-7B-chat',
  'client': <openai.OpenAI at 0x7ff0442ae9d0>}}

In [25]:
#(llmeval) setup@SYS-551A-T-Workstation:/data/2024$ python -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-medium-128k-instruct --max-model-len 4000 --port 8001

In [36]:
ask_llm(client_dict,"RakutenAI-7B-chat","元気?")
#ask_llm(client_dict,"Phi-3-medium-128k-instruct","元気?")

BadRequestError: Error code: 400 - {'object': 'error', 'message': "expected token 'end of print statement', got 's'", 'type': 'BadRequestError', 'param': None, 'code': 400}

In [33]:
q="たぬきに純粋理性批判は理解できますか?"


In [29]:
# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8001/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
q="たぬきに純粋理性批判は理解できますか?"
completion = client.chat.completions.create(
                                      model="microsoft/Phi-3-medium-128k-instruct",
  messages=[
    #{"role": "system", "content": "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"},
    {"role": "user", "content": q}
  ]
)



In [34]:
completion.choices[0].message.content

' たぬきには、人間の高度な思考を理解することができないかもしれません。しかし、簡単な指示や概念を説明することは可能です。純粋理性批判（Critique of Pure Reason）は、近代哲学の中心的な思想の一つであり、多くの人間が理解することが可能です。ただし、たぬきにもわかりやすく説明する必要があります。'

In [16]:
completion

ChatCompletion(id='cmpl-a3200b34280e468a991ff5e49c01cd8c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' はい、大丈夫です。あなたは？', role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1717568033, model='hatakeyama-llm-team/Tanuki-8B-Instruct', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=20, total_tokens=33))